In [1]:
from fomo.models.clip.clip_base import ClipBase
import torch
from PIL import Image

/Users/serghei/Library/Caches/pypoetry/virtualenvs/fomo-2ly0zk_n-py3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
clip = ClipBase()
clip.to_cpu()
clip.eval();

In [3]:
clip.precompute_prompt_features(["a picture of a cat", "a picture of kitten"])

In [4]:
img = Image.open("./cat.jpg")

In [5]:
img_tensor = clip.transform(img).view(1, 3, 224, 224)

In [6]:
with torch.no_grad():
    logits = clip.forward(img_tensor)
print(logits)

tensor([[29.0192, 26.0525]])


In [7]:
torch.functional.F.softmax(logits, dim=1)

tensor([[0.9510, 0.0490]])

In [8]:
with torch.no_grad():
 a = clip.encode_images(img_tensor)

In [9]:
clip._precomputed_prompt_features.shape

torch.Size([2, 512])

In [10]:
logit_scale = clip.logit_scale
image_features = a
text_features = clip._precomputed_prompt_features

In [11]:
o = logit_scale * image_features @ text_features.t()
o -= o.max()
o = torch.exp(o)
o /= o.sum()
o

tensor([[0.9510, 0.0490]])

In [12]:
torch.exp(o - o.max())

tensor([[1.0000, 0.4057]])

In [13]:
with torch.no_grad():
    clip.forward(img_tensor)

In [1]:
from fomo.metrics import METRICS
from fomo.models import Model
from fomo.pipelines.evaluate import EvaluatePipeline
from fomo.utils.data.datasets import DatasetInitializer
from fomo.models.clip.clip_base import ClipBase


model = ClipBase()
dataset_initializers =  [DatasetInitializer.from_str(DatasetInitializer.OXFORD_FLOWERS.name)]

print(dataset_initializers[0].value(
    train=True, 
).dataset)

pipeline = EvaluatePipeline(
    model=model, dataset_initializers=dataset_initializers, metrics=METRICS  # type: ignore
)

results = pipeline.run()

/Users/serghei/Library/Caches/pypoetry/virtualenvs/fomo-2ly0zk_n-py3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Dataset Flowers102
    Number of datapoints: 1020
    Root location: data
    split=train
Dataset:  Dataset Flowers102
    Number of datapoints: 6149
    Root location: data
    split=test
    StandardTransform
Transform: Compose(
               Resize(size=224, interpolation=bicubic, max_size=None, antialias=True)
               CenterCrop(size=(224, 224))
               <function _convert_image_to_rgb at 0x15d2ebc40>
               ToTensor()
               Normalize(mean=(0.48145466, 0.4578275, 0.40821073), std=(0.26862954, 0.26130258, 0.27577711))
           )
Labels:  ['pink primrose', 'hard-leaved pocket orchid', 'canterbury bells', 'sweet pea', 'english marigold', 'tiger lily', 'moon orchid', 'bird of paradise', 'monkshood', 'globe thistle', 'snapdragon', "colt's foot", 'king protea', 'spear thistle', 'yellow iris', 'globe-flower', 'purple coneflower', 'peruvian lily', 'balloon flower', 'giant white arum lily', 'fire lily', 'pincushion flower', 'fritillary', 'red ginger', 'grape

In [1]:
from fomo.models import Model
from fomo.pipelines.precompute_embeddings import PrecomputeEmbeddingsPipeline
from fomo.utils.data.datasets import DatasetInitializer


model = Model.from_str(Model.CLIP_VIT_BASE_PATCH16_PRETRAINED.name)
dataset_initializers =  [DatasetInitializer.from_str(DatasetInitializer.OXFORD_FLOWERS.name)]

pipeline = PrecomputeEmbeddingsPipeline(
    model=model, dataset_initializers=dataset_initializers
)

results = pipeline.run()

/Users/serghei/Library/Caches/pypoetry/virtualenvs/fomo-2ly0zk_n-py3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


{'dataset': (Dataset Flowers102
    Number of datapoints: 6149
    Root location: data
    split=test
    StandardTransform
Transform: <property object at 0x15986a700>, ['pink primrose', 'hard-leaved pocket orchid', 'canterbury bells', 'sweet pea', 'english marigold', 'tiger lily', 'moon orchid', 'bird of paradise', 'monkshood', 'globe thistle', 'snapdragon', "colt's foot", 'king protea', 'spear thistle', 'yellow iris', 'globe-flower', 'purple coneflower', 'peruvian lily', 'balloon flower', 'giant white arum lily', 'fire lily', 'pincushion flower', 'fritillary', 'red ginger', 'grape hyacinth', 'corn poppy', 'prince of wales feathers', 'stemless gentian', 'artichoke', 'sweet william', 'carnation', 'garden phlox', 'love in the mist', 'mexican aster', 'alpine sea holly', 'ruby-lipped cattleya', 'cape flower', 'great masterwort', 'siam tulip', 'lenten rose', 'barbeton daisy', 'daffodil', 'sword lily', 'poinsettia', 'bolero deep blue', 'wallflower', 'marigold', 'buttercup', 'oxeye daisy', '

TypeError: default_collate: batch must contain tensors, numpy arrays, numbers, dicts or lists; found <class 'torchvision.datasets.flowers102.Flowers102'>

In [ ]:
from fomo.models import Model
from fomo.pipelines.train import TrainPipeline
from fomo.utils.data.datasets import DatasetInitializer

model = Model.from_str(Model.CLIP_VIT_BASE_PATCH16_PRETRAINED.name)
dataset_initializers = [DatasetInitializer.from_str(DatasetInitializer.CIFAR10.name)]

print(model.__dict__)

pipeline = TrainPipeline(
    model=model,
    dataset_initializers=dataset_initializers,
    epochs=1,
    lr=1e-5,
    val_split=0.1,
)

results = pipeline.run()

In [1]:
import os
import torch
import clip
from torch.utils.data import DataLoader, random_split, Subset
from torchvision.datasets import CIFAR10
from torchvision import transforms
from PIL import Image
from tqdm import tqdm
from fomo.models.adapters.simple_mlp_adapter import SimpleMLPAdapter
from fomo.utils.data.datasets import DatasetInitializer
from fomo.models.clip.clip_with_adapter import ClipWithAdapter

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")


model = ClipWithAdapter(
    adapter=SimpleMLPAdapter(1024, 100),
)

transforms = model._clip.transforms

# Download the dataset
train_dataset = CIFAR10(root="data", train=True, download=True, transform=transforms)
test_dataset =  CIFAR10(root="data", train=False, download=True, transform=transforms)

# Define subset sizes
total_subset_size = 100  # Total number of samples to use
train_subset_size = int(0.8 * total_subset_size)
validation_subset_size = total_subset_size - train_subset_size

# Create subsets
indices = torch.randperm(len(train_dataset))[:total_subset_size]
train_indices = indices[:train_subset_size]
validation_indices = indices[train_subset_size:]

train_subset = Subset(train_dataset, train_indices)
validation_subset = Subset(train_dataset, validation_indices)

# Data loaders for the subsets
train_loader = DataLoader(train_subset, batch_size=1, shuffle=True)
validation_loader = DataLoader(validation_subset, batch_size=1, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)


loss_fn = torch.nn.CrossEntropyLoss()

# Optimizer
# Partly inspired from https://github.com/openai/CLIP/issues/83
optimizer = torch.optim.Adam(model.parameters(), lr=5e-5,betas=(0.9,0.98),eps=1e-6,weight_decay=0.2)

# Training Loop
# for epoch in range(3): 
total_loss = 0
model._clip.eval()
for images, labels in tqdm(train_loader):
    prompts = [f"a photo of a {train_dataset.classes[c]}" for c in labels]

    logits_per_image = model.forward(images, prompts)

    loss = loss_fn(logits_per_image, labels)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    total_loss += loss.item()

avg_loss = total_loss / len(train_loader)
    # print(f"Epoch {epoch+1}, Loss: {avg_loss}")

    # # Validation
    # model.eval()
    # with torch.no_grad():
    #     validation_loss = 0
    #     correct = 0
    #     total = 0
    #     for images, labels in validation_loader:

    #         validation_loss += loss.item()

    #         # Calculate training accuracy
    #         _, predicted = logits_per_image.max(1)
    #         total += labels.size(0)
    #         correct += predicted.eq(targets).sum().item()


    #     avg_val_loss = validation_loss / len(validation_loader)
    #     avg_val_accuracy = 100. * correct / total

    #     print(f"Validation Loss: {avg_val_loss}")



/Users/serghei/Library/Caches/pypoetry/virtualenvs/fomo-2ly0zk_n-py3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: cpu
Files already downloaded and verified
Files already downloaded and verified


 39%|███▉      | 31/80 [14:02<22:11, 27.18s/it]


KeyboardInterrupt: 